# ESM3 Tests

In [ ]:
# from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, GenerationConfig
from esm.sdk import client

In [7]:
model = client(model="esm3-small-2024-08", url="https://forge.evolutionaryscale.ai", token="")

In [19]:
sequence = "FVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYC"

In [20]:
structure_prediction_config = GenerationConfig(
    track="structure",  # We want ESM3 to generate tokens for the structure track
    num_steps=8,
    temperature=0.7,
)

structure_prediction_prompt = ESMProtein(sequence=sequence)

structure_prediction = model.generate(
    structure_prediction_prompt, structure_prediction_config
)

In [ ]:
# structure_prediction

In [22]:
structure_prediction_chain = structure_prediction.to_protein_chain()

In [ ]:
structure_prediction_chain.to_pdb_string()

In [ ]:
## Use py3Dmol to visualize the structure
import py3Dmol

view = py3Dmol.view(width=800, height=800)
view.addModel(structure_prediction_chain.to_pdb_string(), "pdb")
view.setStyle({"cartoon": {"color": "spectrum"}})
view.zoomTo()
view.show()

## Multimer

In [2]:
from esm.sdk.api import ESMProtein, GenerationConfig
from esm.sdk import client

c:\ProgramData\pymol\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = client(model="esm3-medium-multimer-2024-09", url="https://forge.evolutionaryscale.ai", token="2aAJB9I9l4eR1ja1X9l4HQ")

In [27]:
sequence = "FVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYC|VFPSSVFVPDEWEVSREKITLLRELGQGSFGMVYEGNARDIIKGEAETRVAVKTVNESASLRERIEFLNEASVMKGFTCHHVVRLLGVVSKGQPTLVVMELMAHGDLKSYLRSLRPEAENNPGRPPPTLQEMIQMAAEIADGMAYLNAKKFVHRDLAARNCMVAHDFTVKIGDFGMTRDIYETDYYRKGGKGLLPVRWMAPESLKDGVFTTSSDMWSFGVVLWEITSLAEQPYQGLSNEQVLKFVMDGGYLDQPDNCPERVTDLMRMCWQFNPKMRPTFLEIVNLLKDDLHPSFPEVSFFHSEENK"

In [28]:
structure_prediction_config = GenerationConfig(
    track="structure",  # We want ESM3 to generate tokens for the structure track
    num_steps=8,
    temperature=0.7,
)

structure_prediction_prompt = ESMProtein(sequence=sequence)

In [29]:
structure_prediction = model.generate(
    structure_prediction_prompt, structure_prediction_config
)

In [12]:
structure_prediction_complex = structure_prediction.to_protein_complex()

In [19]:
structure_prediction_complex.to_pdb('./test_complex.pdb')

C:\Users\Colby\AppData\Roaming\Python\Python310\site-packages\esm\utils\structure\protein_complex.py:223: UserWarning: Entity ID not found in metadata, using None as default
  warnings.warn("Entity ID not found in metadata, using None as default")


## Visualize

In [20]:
## Use py3Dmol to visualize the structure
import py3Dmol

view = py3Dmol.view(width=800, height=800)
view.addModel(structure_prediction_complex.to_pdb_string(), "pdb")
# view.addModel(folded_protein.to_pdb_string(), "pdb")
# view.setStyle({"cartoon": {"color": "spectrum"}})
## color by chain
view.setStyle({"cartoon": {"color": "blue"}, "chain": "A"})
view.setStyle({"cartoon": {"color": "green"}, "chain": "B"})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.